In [3]:
from ultralytics import YOLO
import cv2
import numpy as np

def load_and_predict(model, image_path):
    """Run YOLO model on the input image and return predictions."""
    results = model.predict(source=image_path, save=TRUE)
    return results

def get_fruit_features(results, model):
    """Extract bounding boxes, labels, and colors from YOLO predictions."""
    fruits = []
    for result in results:
        for box, cls in zip(result.boxes.xywh, result.boxes.cls):
            x, y, w, h = box[:4]  # Bounding box (center x, center y, width, height)
            class_name = model.names[int(cls)]  # Class name
            fruits.append({
                'bbox': (x, y, w, h),
                'class': class_name,
                'color': get_color(class_name),  # Map class name to color
            })
    return fruits

def get_color(class_name):
    """Map fruit class to color."""
    color_map = {
        'apple': 'red',
        'banana': 'yellow',
        'orange': 'orange',
        # Add more classes and colors as needed
    }
    return color_map.get(class_name, 'unknown')

def match_fruits(front_fruits, back_fruits, iou_threshold=0.5):
    """Match fruits between front and back images using IoU."""
    matched = []
    for fruit1 in front_fruits:
        for fruit2 in back_fruits:
            iou = calculate_iou(fruit1['bbox'], fruit2['bbox'])
            if iou > iou_threshold:
                matched.append(fruit1)
                break
    return matched

def calculate_iou(box1, box2):
    """Calculate Intersection over Union (IoU) between two bounding boxes."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Convert (x, y, w, h) to (x1, y1, x2, y2)
    box1 = [x1 - w1 / 2, y1 - h1 / 2, x1 + w1 / 2, y1 + h1 / 2]
    box2 = [x2 - w2 / 2, y2 - h2 / 2, x2 + w2 / 2, y2 + h2 / 2]

    # Calculate the intersection
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    inter_area = max(0, xB - xA) * max(0, yB - yA)

    # Calculate the union
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

def count_fruits_by_color(fruits):
    """Count fruits grouped by color."""
    color_count = {}
    for fruit in fruits:
        color = fruit['color']
        color_count[color] = color_count.get(color, 0) + 1
    return color_count

# Paths to images and YOLO model weights
front_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\front.jpg'
back_image = r'C:\Users\saart\OneDrive\Desktop\UAS_DTU_Round_2_Task_data\8\back.jpg'
model_path = 'best.pt'

# Load YOLO model
model = YOLO('best.pt')

# Predict on front and back images
front_results = load_and_predict(model, front_image)
back_results = load_and_predict(model, back_image)

# Extract fruit features
front_fruits = get_fruit_features(front_results, model)
back_fruits = get_fruit_features(back_results, model)

# Match fruits and count
matched_fruits = match_fruits(front_fruits, back_fruits)
unique_fruits = front_fruits + [fruit for fruit in back_fruits if fruit not in matched_fruits]
fruit_count = count_fruits_by_color(unique_fruits)

print("Fruit Count by Color:", fruit_count)


ImportError: cannot import name 'YOLOv8' from 'ultralytics' (C:\Users\saart\anaconda3\Lib\site-packages\ultralytics\__init__.py)